In [3]:
import requests
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from requests import get
from bs4 import BeautifulSoup as BS
import pandas as pd
Website="mouthshut"
Reviews=[]
Label=[]
Rating=[]
Title=[]

In [4]:
def mouthshut(str):
    soup = BS(requests.get(str).text, "html.parser")
    ste =' '
    for title in soup.select("a[id^=ctl00_ctl00_ContentPlaceHolderFooter_ContentPlaceHolderBody_rptreviews_]"):
        #print(title)
        items = title.get('href')
        if(items!=None):
            response=requests.get(items)
            content=response.text
            soup2=BS(content)
            for i in soup2.findAll('div', attrs={"class":'rev-main-content'}):
                for va in i.findAll('p'):
                    ste=ste+va.text
                #review=i.p.text
                Reviews.append(ste)
                #print(ste)
                ste=" "
    start=-2
    for var in soup.findAll('div', attrs = {'class':'rating'}):
        if start>=0:
            count=0
            for bas in var.findAll('i', attrs = {'class':'icon-rating rated-star'}):
                count+=1
            Rating.append(count)
            #print(count)
        start+=1
    for var in soup.findAll('strong'):
        for s in var.find_all('a'):
            Title.append(s.text)
            #print(s.text)
    for var in soup.findAll('li', attrs = {'class':'next'}):
            str2=var.a['href']
            mouthshut(str2)


In [5]:
def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
      
    #print("Overall sentiment dictionary is : ", sentiment_dict) 
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
    #
    #Label.append(sentiment_dict['neg']*100*-1) 
    #Label.append(sentiment_dict['neu']*100) 
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
  
    #print("Sentence Overall Rated As", end = " ") 
  
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
         Label.append((sentiment_dict['pos']*100))
  
    elif sentiment_dict['compound'] <= - 0.05 : 
         Label.append(sentiment_dict['neg']*100*-1)
  
    else : 
        Label.append(0) 

In [6]:
#call from googlescrap file
def callfunc(str):
    global Reviews
    global Label
    global Rating
    global Title
    
    Reviews=[]
    Label=[]
    Rating=[]
    Title=[]

    mouthshut(str)
    #for review in Reviews:
    #    sentiment_scores(review)
    #print(len(Reviews))
    #print(len(Title))
    #print(len(Rating))
    #print(len(Label))
    
    #df = pd.DataFrame({'Website':Website,"Placements":None,"Faculty":None,"Infrastructure":None,"Rating":None,"Other":None,'Review':Reviews,'Label':Label}) 
    df=pd.DataFrame(columns=('Website','Title','Review','Placement','Infrastructure','Faculty','Social','Other','Label','Rating'))
    df=pd.DataFrame({'Website':Website,'Title':Title,'Review':Reviews,'Placement':None,'Infrastructure':None,'Faculty':None,'Social':None,'Other':None,'Label':None,'Rating':Rating})
    #df.to_csv('IITD_mouthshut.csv', index=False, encoding='utf-8')
    return(df)

In [7]:
#callfunc("https://www.mouthshut.com/product-reviews/Jaypee-Institute-of-Information-Technology-Noida-reviews-925771668")